In [ ]:
!pip install transformers nltk deep-translator torch



In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
from nltk.metrics.distance import edit_distance
from nltk.tokenize import word_tokenize
from deep_translator import GoogleTranslator
import re


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sinhala_vocab.txt to sinhala_vocab.txt


In [ ]:
#function to load vocabulary from the text file
def load_vocab(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f]

#example
file_path = "/content/sinhala_vocab.txt"
sinhala_vocab = load_vocab(file_path)
print(sinhala_vocab)


['අකුරු', 'අඩවිය', 'අනුරාධපුර', 'අභිමානය', 'අරවින්ද', 'අවශ්\u200dයතාව', 'අහස', 'අළුත්', 'ඇඳ', 'ඇඳුම්', 'ඇඩ්වෝකේට්', 'ඇනත්', 'ඇය', 'ඇල', 'ඉඳි', 'ඉඩම', 'ඉන්ද්\u200dරජාල', 'ඉස්කෝල', 'ඉස්සර', 'ඉහළ', 'ඉවරය', 'ඉස්තර', 'ඉන්ධන', 'ඉන', 'ඉක්මනින්', 'ඉදිරියට', 'උදව්', 'උදාර', 'උදේ', 'උපන්දිනය', 'උපකාරක', 'උපන්සැරිය', 'උපුල්', 'උරහිස්', 'එක', 'එක්ව', 'එක්ක', 'එල', 'එළිය', 'එළවන', 'එසැණින්', 'එහා', 'ඔබ', 'ඔරලෝසුව', 'ඔරුව', 'ඔව්', 'ඕන', 'කතා', 'කටු', 'කටුක', 'කණ්', 'කඳු', 'කඳුළු', 'කඳවුර', 'කමටු', 'කම්මැලි', 'කය', 'කලාපය', 'කොටුව', 'කෝවිල', 'ගඟ', 'ගම', 'ගැටලු', 'ගිණි', 'ගිම්හාන', 'ගුරු', 'ගුරුවර', 'ගොවි', 'ගොඩනැගිල්ල', 'ගෝඨා', 'ගෘහය', 'ගෘහස්ථ', 'චතුර', 'චාරිකාව', 'චිත්\u200dර', 'චිත්ත', 'චැටර්', 'චීන', 'චෙස්', 'ජනප්\u200dරිය', 'ජලය', 'ජීවිතය', 'ජය', 'ජලමඟ', 'ටික', 'ඩොක්ටර්', 'ඩිජිටල්']


In [ ]:
#load vocabulary
vocab_file = "/content/sinhala_vocab.txt"
sinhala_vocab = load_vocab(vocab_file)

#spelling correction function
def correct_typo(word, vocab):
    corrections = [(v, edit_distance(word, v)) for v in vocab]
    corrections.sort(key=lambda x: x[1])
    return corrections[0][0] if corrections[0][1] <= 2 else word  # Adjust threshold as needed

#example
input_word = "අනුරාදපුර"  #misspelled
corrected_word = correct_typo(input_word, sinhala_vocab)
print(f"Input Word: {input_word}")
print(f"Corrected Word: {corrected_word}")

Input Word: අනුරාදපුර
Corrected Word: අනුරාධපුර


In [ ]:
input_word = "අබිමානය"  #misspelled
corrected_word = correct_typo(input_word, sinhala_vocab)
print(f"Input Word: {input_word}")
print(f"Corrected Word: {corrected_word}")

Input Word: අබිමානය
Corrected Word: අභිමානය


In [ ]:
input_word = "එසැනින්"  #misspelled
corrected_word = correct_typo(input_word, sinhala_vocab)
print(f"Input Word: {input_word}")
print(f"Corrected Word: {corrected_word}")

Input Word: එසැනින්
Corrected Word: එසැණින්


In [ ]:
from sklearn.metrics import accuracy_score

#load vocabulary
vocab_file = "/content/sinhala_vocab.txt"
sinhala_vocab = load_vocab(vocab_file)

#spelling correction function
def correct_typo(word, vocab):
    corrections = [(v, edit_distance(word, v)) for v in vocab]
    corrections.sort(key=lambda x: x[1])
    return corrections[0][0] if corrections[0][1] <= 2 else word  # Adjust threshold as needed

#test dataset with misspelled and correct word pairs
test_dataset = [
    ("අනුරාදපුර", "අනුරාධපුර"),  #misspelled,corrected
    ("අබිමානය", "අභිමානය"),
    ("එසැනින්", "එසැණින්"),
]

#function to evaluate accuracy
def evaluate_accuracy(test_dataset, vocab):
    y_true = []  #correct words
    y_pred = []  #predicted words

    for misspelled, correct_word in test_dataset:
        predicted_word = correct_typo(misspelled, vocab)
        y_true.append(correct_word)
        y_pred.append(predicted_word)

    #calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

#example usage
accuracy = evaluate_accuracy(test_dataset, sinhala_vocab)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


In [ ]:
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00


In [ ]:
import Levenshtein

#phonetic Mapping for Sinhala
phonetic_map = {
    'ක': 'K', 'ග': 'G', 'ඛ': 'K', 'ඝ': 'G',  # Guttural sounds
    'ච': 'C', 'ජ': 'J', 'ඡ': 'C', 'ඣ': 'J',  # Palatal sounds
    'ට': 'T', 'ඩ': 'D', 'ඨ': 'T', 'ඪ': 'D',  # Retroflex sounds
    'ත': 'T', 'ද': 'D', 'ථ': 'T', 'ධ': 'D',  # Dental sounds
    'ප': 'P', 'බ': 'B', 'භ': 'B', 'ඵ': 'P',  # Labial sounds
    'ම': 'M', 'ය': 'Y', 'ර': 'R', 'ල': 'L',  # Nasals and liquids
    'ව': 'V', 'ස': 'S', 'හ': 'H', 'ෂ': 'S',  # Fricatives and others
    '෴': 'Z', 'ඞ': 'N', 'ඤ': 'N', 'ඵ': 'P'   # Rare/less common sounds
}

#normalize word by removing vowels and modifiers
def normalize(word):
    vowels_modifiers = ['අ', 'ඇ', 'ඉ', 'එ', 'උ', 'ඔ', 'ා', 'ැ', 'ි', 'ී', 'ු', 'ූ', 'ෙ', 'ේ', 'ෟ']
    return ''.join([char for char in word if char not in vowels_modifiers])

#generate phonetic code for a word
def sinhala_phonetic(word):
    word = normalize(word)
    phonetic_code = "".join([phonetic_map.get(char, '') for char in word if char in phonetic_map])
    return phonetic_code

#load the vocabulary and generate phonetic codes
def load_vocab_with_phonetics(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        vocab = [line.strip() for line in f]
    return {word: sinhala_phonetic(word) for word in vocab}

#find the closest match based on phonetic code and Levenshtein distance
def find_closest_match(input_word, vocab_with_phonetics):
    input_phonetic = sinhala_phonetic(input_word)

    closest_match = None
    min_distance = float('inf')

    #find the word with the closest phonetic code
    for word, phonetic in vocab_with_phonetics.items():
        phonetic_distance = Levenshtein.distance(input_phonetic, phonetic)

        #calculate the Levenshtein distance on the original words as well
        word_distance = Levenshtein.distance(input_word, word)

        #combine the two distances (use a weighted sum or choose based on your preference)
        total_distance = phonetic_distance + word_distance

        if total_distance < min_distance:
            min_distance = total_distance
            closest_match = word

    return closest_match if closest_match else input_word

#example
vocab_path = "/content/sinhala_vocab.txt"

#load vocabulary and generate phonetics
vocab_with_phonetics = load_vocab_with_phonetics(vocab_path)

#test with misspelled words
test_words = ["අඪවිය", "ගලාපය", "උතාර", "ඉහර"]
for word in test_words:
    corrected = find_closest_match(word, vocab_with_phonetics)
    print(f"Word: {word}, Corrected: {corrected}")


Word: අඪවිය, Corrected: අඩවිය
Word: ගලාපය, Corrected: කලාපය
Word: උතාර, Corrected: උදාර
Word: ඉහර, Corrected: ඉහළ


In [ ]:
from sklearn.metrics import accuracy_score

#misspelled words and their correct versions
test_dataset = [
    ("අඪවිය", "අඩවිය"),  #misspelled,corrected
    ("ගලාපය", "කලාපය"),
    ("උතාර", "උදාර"),
    ("ඉහර", "ඉහළ")
]

#function to evaluate accuracy
def evaluate_accuracy(test_dataset, vocab_with_phonetics):
    y_true = []  #correct words
    y_pred = []  #predicted words

    for misspelled, correct_word in test_dataset:
        predicted_word = find_closest_match(misspelled, vocab_with_phonetics)
        y_true.append(correct_word)
        y_pred.append(predicted_word)

    #calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

#example
accuracy = evaluate_accuracy(test_dataset, vocab_with_phonetics)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%
